## **Mapeamento Galp**

##### **___Informações___**

**Projeto:** Galp

**Autor:** Lucas Pimenta dos Santos Monteiro

**Data:** 24/02/2025

##### **___Objetivo___**

A ideia desse cógido é automatizar a visualização de uma imagem .svg gerada no Inkscape de acordo com o output do programa (report.csv) e os dados de entrada definidos (entry_data.csv).

Para isso foram criados rótulos no Inkscape que identificam cada elemento de texto a partir do inkscape:label. Cada label recebeu o nome equivalente a variável proveniente do report.csv e entry_data.csv que representa o valor desejado.

##### **Importando as Bibliotecas**

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

In [2]:
entry_data = pd.read_csv("entry_data.csv")
report = pd.read_csv("report.csv")

##### **Verificando os Dados**

In [3]:
entry_data.head()

,Entry data:,Unnamed: 1,Unnamed: 2
0,Entry feed mass flow rate =,600.0000000000,kg/h
1,Vacuum pressure =,-495.8300000000,mbar
2,Membrane area =,12.9600000000,m²
3,Membrane thickness =,100.0000000000,microns
4,Membrane porosity =,85.0000000000%,NaN


In [4]:
report.head()

,Time (s),Desal - Coolant mass flow rate (kg/h),Desal - Hot feedwater inlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Feed membrane temperature (°C),Desal - Membrane gap temperature (°C),Desal - Gap film boundary temperature (°C),Desal - Film wall temperature (°C),...,RO Module - Mass of water treated (kg),RO Pump - Outlet pressure (bar),RO Pump - Electricity consumption (kWh),RO Feedtank - Mass outflow rate (kg/h),RO Feedtank - Salinity (wt%),RO Feedtank - Temperature (°C),Overall - Net water production (L),Overall - Salinity of the treated water (wt%),Overall - Net electricity generation (kWh),Overall - Net heat recovered (kWh)
0,126.0,600.0,60.002262,35.318529,30.173601,54.855806,47.164965,45.973914,44.702002,43.337233,...,6.482446,51.121526,0.035,616.8,1.004179,25.0,7.018504,0.005048,0.717809,0.539239
1,252.0,600.0,61.104572,35.449641,30.156956,55.810470,47.765640,46.550625,45.252565,43.835558,...,12.957082,51.123110,0.070,616.8,1.008342,25.0,14.059027,0.005064,1.436208,1.067087
2,378.0,600.0,62.154797,35.573314,30.142618,56.722809,48.337477,47.100802,45.779081,44.312337,...,19.423942,51.124687,0.105,616.8,1.012489,25.0,21.120544,0.005080,2.155191,1.584189
3,504.0,600.0,63.155748,35.690063,30.130276,57.594809,48.882071,47.625787,46.282619,44.768502,...,25.883056,51.126259,0.140,616.8,1.016620,25.0,28.202048,0.005096,2.874752,2.091149
4,630.0,600.0,64.110151,35.800370,30.119660,58.428442,49.400964,48.126896,46.764255,45.204999,...,32.334459,51.127825,0.175,616.8,1.020736,25.0,35.302555,0.005112,3.594884,2.588537


##### **Definindo as Colunas que serão utilizadas de cada dataframe**

In [5]:
userows_entry_data = [
    "Cooling heat exchanger flow rate =",
    "Brine flow rate out of the regulating tank =",
    "Temperature at the entrance of the cold side =",
    "Motor power demanded/generated =",                                     
    "Electrolyzer cell temperature =",                                      
    "Cathode pressure =",
    "RO feedtank brackish water temperature =",
    "RO feedtank brackish water salinity =",
    "RO pump power ="
]

usecols_report = [
"Time (s)",
"Overall - Net water production (L)",
"Desal - Distilled water production rate (kg/h)",
"FeedTank - Brackish water inlet flow rate (kg/h)",
"ColdHex - Heat transfer rate (W)",
"ColdHex - Cold side outlet temperature (°C)",
"Desal - Hot feedwater outlet temperature (°C)",
"Desal - Hot feedwater outlet mass flow rate (kg/h)",
"Desal - Hot feedwater outlet salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"ColdHex - Hot side outlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"HotHex - Cold side salinity (wt%)",
"HotHex - Heat transfer rate (W)",
"HotHex - Hot side outlet temperature (°C)",
"HotHex - Hot side inlet temperature (°C)",
"HotHex - Cold side outlet temperature (°C)",
"ColdHex - Hot side inlet temperature (°C)",
"ColdHex - Hot side inlet mass flow rate (kg/h)",
"HotHex - Cold side inlet temperature (°C)",
"Time (s)",
"SolarField - Solar irradiance (W/m²)", # segundo 11088 -> tempo de máximo
"SolarField - Efficiency",
"SolarField - Solar field inlet temperature (°C)",
"SolarField - Solar field outlet temperature (°C)",
"CircuitFromSolar - Outlet temperature (°C)",
"CircuitToSolar - Inlet temperature (°C)",
"ColdHex - Hot side salinity (wt%)",
"FeedTank - Water salinity (wt%)",
"FeedTank - Water temperature (°C)",
"Desal - Film wall temperature (°C)",
"Desal - Gap film boundary temperature (°C)",
"HCPV - Inlet temperature (°C)",
"HCPV - Outlet temperature (°C)",
"HCPV - Mass flow rate (kg/s)",
"HCPV - Heat transfer rate recovered from the solar cells (kW)",
"HCPV - Electrical power (kW)",
"HCPV - Global panel electrical efficiency (%)",
"HCPV - Electrical energy generated (kWh)",
"CircuitToSolar - Mass flow rate (kg/s)",
"ETC - Efficiency",
"ETC - ETC inlet temperature (°C)",
"ETC - ETC outlet temperature (°C)",
"ETC - Mass flow rate (kg/s)",
"PVT - PVT inlet temperature (°C)",
"PVT - Thermal Efficiency",
"PVT - Electrical power output (W)",
"PVT - Electrical Efficiency",
"PVT - Electrical energy generated (kWh)",
"PVT - PVT outlet temperature (°C)",
"PVT - Mass flow rate (kg/s)",
"PVT - Heat transfer rate (W)",
"ETC - Heat transfer rate (W)",
"SolarField - Heat transfer rate (W)",
"Reservoir - Temperature (°C)",
"Mix Valve - Mass flow rate in from reservoir(kg/s)",
"Mix Valve - Temperature in from reservoir(kg/s)",
"Mix Valve - Out Temperature (°C)",
"Mix Valve - Out Mass flow rate (kg/s)",
"Motor - Outlet mass flow rate (kg/s)",
"Motor - Outlet temperature motor (°C)",
"Motor - Radiator in mass flow rate (kg/s)",
"Motor - Radiator bypass mass flow rate (kg/s)",
"Motor - Recirculation mass flow rate (kg/s)",
"Motor - Nominal Mass flow rate (kg/s)",
"Motor - Mixture temperature engine (°C)",
"Motor - Mixture temperature radiator (°C)",
"Motor - Oulet temperature radiator (°C)",
"Motor - Heat coolant (kW)",
"Motor - System efficiency",
"Motor - Electrical energy generated/demanded (kWh)",
"Motor - Fuel consumed (L)",
"Motor - Fuel consumption (L/h)",
"Motor - Fuel consumption (g/kWh)",
"Electrolyzer - Hydrogen flow rate (Nm³/h)",
"Electrolyzer - Hydrogen produced (Nm³)",
"Electrolyzer - Water consumption rate (kg/h)",
"Electrolyzer - Water consumed (kg)",
"Electrolyzer - Conversion efficiency (%)",
"Electrolyzer - Heat transfer rate generated (W)",
"Electrolyzer - Power demand (W)",
"Electrolyzer - Electricity demand (kWh)",
"RO Feedtank - Temperature (°C)",
"RO Module - Brine outlet salinity (wt%)",
"RO Module - Brine outflow rate (kg/h)",
"RO Feedtank - Mass outflow rate (kg/h)",
"RO Feedtank - Salinity (wt%)",
"RO Pump - Outlet pressure (bar)",
"RO Module - Permeate salinity (wt%)",
"RO Pump - Electricity consumption (kWh)",
"Mix Valve - Mass flow rate in from central reservoir(kg/s)",
"Mix Valve - Mass flow rate in from central reservoir(kg/s)",
"Mix Valve - Temperature in from central reservoir(kg/s)",
"Central Reservoir - Temperature (°C)",
"Overall - Net electricity generation (kWh)",
"Overall - Net heat recovered (kWh)",
"Overall - Salinity of the treated water (wt%)",
"Motor - Heat transfer rate (kW)"
]

##### Filtrando os Dataframes de Acordo com as Colunas

In [6]:
entry_data_filtrado = entry_data[entry_data['Entry data:'].isin(userows_entry_data)]
entry_data_filtrado

,Entry data:,Unnamed: 1,Unnamed: 2
19,Cooling heat exchanger flow rate =,2000.0000000000,kg/h
20,Temperature at the entrance of the cold side =,30.0000000000,°C
25,Brine flow rate out of the regulating tank =,0.0000000000,kg/h
127,Motor power demanded/generated =,20.0000000000,kW
143,Electrolyzer cell temperature =,40.0000000000,°C
145,Cathode pressure =,70.0000000000,bar
182,RO pump power =,1.0000000000,kW
186,RO feedtank brackish water temperature =,25.0000000000,°C
187,RO feedtank brackish water salinity =,1.0000000000,wt%


In [7]:
report_filtrado = report.filter(items=usecols_report)
report_filtrado.head()

,Time (s),Overall - Net water production (L),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,RO Pump - Outlet pressure (bar),RO Module - Permeate salinity (wt%),RO Pump - Electricity consumption (kWh),Mix Valve - Mass flow rate in from central reservoir(kg/s),Mix Valve - Temperature in from central reservoir(kg/s),Central Reservoir - Temperature (°C),Overall - Net electricity generation (kWh),Overall - Net heat recovered (kWh),Overall - Salinity of the treated water (wt%),Motor - Heat transfer rate (kW)
0,126.0,7.018504,17.631971,17.631971,12868.324185,35.537162,35.318529,582.368029,1.036216,1.005765,...,51.121526,0.005449,0.035,0.5,53.647965,53.647965,0.717809,0.539239,0.005048,13.248403
1,252.0,14.059027,18.489083,18.489083,11639.024801,35.008094,35.449641,581.510917,1.044015,1.011844,...,51.123110,0.005478,0.070,0.5,54.813649,54.813649,1.436208,1.067087,0.005064,13.248403
2,378.0,21.120544,19.315987,19.315987,10579.939478,34.552300,35.573314,580.684013,1.052100,1.018230,...,51.124687,0.005506,0.105,0.5,55.930081,55.930081,2.155191,1.584189,0.005080,13.248403
3,504.0,28.202048,20.113255,20.113255,9668.223014,34.159941,35.690063,579.886745,1.060462,1.024913,...,51.126259,0.005535,0.140,0.5,57.000089,57.000089,2.874752,2.091149,0.005096,13.248403
4,630.0,35.302555,20.881605,20.881605,8884.064269,33.822487,35.800370,579.118395,1.069094,1.031887,...,51.127825,0.005564,0.175,0.5,58.026310,58.026310,3.594884,2.588537,0.005112,13.248403


#### Separando o tempo final para o overall results

In [8]:
report_filtrado_tempo_final = report_filtrado.tail(1)
report_filtrado_tempo_final.head()

,Time (s),Overall - Net water production (L),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,RO Pump - Outlet pressure (bar),RO Module - Permeate salinity (wt%),RO Pump - Electricity consumption (kWh),Mix Valve - Mass flow rate in from central reservoir(kg/s),Mix Valve - Temperature in from central reservoir(kg/s),Central Reservoir - Temperature (°C),Overall - Net electricity generation (kWh),Overall - Net heat recovered (kWh),Overall - Salinity of the treated water (wt%),Motor - Heat transfer rate (kW)
199,25200.0,1367.921197,28.457817,28.457817,4341.912039,31.868004,37.040699,571.542183,3.881276,3.697188,...,51.371533,0.010847,7.0,0.5,69.349094,69.349094,146.383964,51.038063,0.009287,13.248403


In [9]:
# Net electricity generation

net_electricity_generation = str(round(float(report_filtrado_tempo_final['Overall - Net electricity generation (kWh)'].iloc[0]), 2)) + " kWh"
print("Net electricity generation:", net_electricity_generation)

# Net water production -> tem que colocar no valor do Overall - Net water production - ft - (L) no svg

net_water_production = str(round(float(report_filtrado_tempo_final['Overall - Net water production (L)'].iloc[0]), 2)) + " L"
print("Net water production:", net_water_production)

# Net heat recovered

net_heat_recovered = str(round(float(report_filtrado_tempo_final['Overall - Net heat recovered (kWh)'].iloc[0]), 2)) + " kWh"
print("Net heat recovered:", net_heat_recovered)


Net electricity generation: 146.38 kWh
Net water production: 1367.92 L
Net heat recovered: 51.04 kWh


##### Separando apenas o tempo de máxima radiação solar do output gerado no report.csv

In [10]:
report_filtrado.tail()

,Time (s),Overall - Net water production (L),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,RO Pump - Outlet pressure (bar),RO Module - Permeate salinity (wt%),RO Pump - Electricity consumption (kWh),Mix Valve - Mass flow rate in from central reservoir(kg/s),Mix Valve - Temperature in from central reservoir(kg/s),Central Reservoir - Temperature (°C),Overall - Net electricity generation (kWh),Overall - Net heat recovered (kWh),Overall - Salinity of the treated water (wt%),Motor - Heat transfer rate (kW)
195,24696.0,1342.861368,29.337522,29.337522,4378.321791,31.883670,37.138524,570.662478,3.824769,3.637754,...,51.367327,0.010740,6.860,0.5,70.389147,70.389147,143.587159,50.541447,0.009197,13.248403
196,24822.0,1349.143197,29.127651,29.127651,4370.074994,31.880122,37.115501,570.872349,3.839124,3.652749,...,51.368378,0.010767,6.895,0.5,70.142915,70.142915,144.287633,50.671611,0.009219,13.248403
197,24948.0,1355.413920,28.910848,28.910848,4361.252334,31.876326,37.091506,571.089152,3.853328,3.667656,...,51.369429,0.010793,6.930,0.5,69.887287,69.887287,144.987265,50.797723,0.009242,13.248403
198,25074.0,1361.673321,28.687460,28.687460,4351.861917,31.872285,37.066565,571.312540,3.867379,3.682470,...,51.370481,0.010820,6.965,0.5,69.622581,69.622581,145.686045,50.919851,0.009264,13.248403
199,25200.0,1367.921197,28.457817,28.457817,4341.912039,31.868004,37.040699,571.542183,3.881276,3.697188,...,51.371533,0.010847,7.000,0.5,69.349094,69.349094,146.383964,51.038063,0.009287,13.248403


In [11]:
report_filtrado = report_filtrado.iloc[[report_filtrado['SolarField - Solar irradiance (W/m²)'].idxmax()]].reset_index(drop=True)
report_filtrado.head()

,Time (s),Overall - Net water production (L),Desal - Distilled water production rate (kg/h),FeedTank - Brackish water inlet flow rate (kg/h),ColdHex - Heat transfer rate (W),ColdHex - Cold side outlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Hot feedwater outlet mass flow rate (kg/h),Desal - Hot feedwater outlet salinity (wt%),HotHex - Cold side salinity (wt%),...,RO Pump - Outlet pressure (bar),RO Module - Permeate salinity (wt%),RO Pump - Electricity consumption (kWh),Mix Valve - Mass flow rate in from central reservoir(kg/s),Mix Valve - Temperature in from central reservoir(kg/s),Central Reservoir - Temperature (°C),Overall - Net electricity generation (kWh),Overall - Net heat recovered (kWh),Overall - Salinity of the treated water (wt%),Motor - Heat transfer rate (kW)
0,11088.0,629.161715,32.206479,32.206479,4467.770462,31.922157,37.297,567.793521,2.144427,2.029319,...,51.243514,0.007855,3.08,0.5,76.268382,76.268382,64.717439,27.424745,0.006804,6.818403


##### Lendo o arquivo do Inkscape e definindo uma função de substituição de valores

In [12]:
# Caminho para o arquivo SVG
arquivo_svg = 'Diag_geral_2025_07_01.svg'

# Parse o arquivo SVG
tree = ET.parse(arquivo_svg)
root = tree.getroot()

# Definir namespaces
namespaces = {'svg': 'http://www.w3.org/2000/svg', 'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

def substituir_texto_por_label(label, novo_texto):
    # Encontre todos os elementos do tipo 'text'
    textos = root.findall('.//svg:text', namespaces)
    
    # Para cada elemento 'text' encontrado
    for texto in textos:
        # Verifique se o elemento 'text' possui o atributo 'inkscape:label'
        if texto.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label') == label:
            # Itere sobre os elementos 'tspan' dentro do elemento 'text'
            for tspan in texto.findall('.//svg:tspan', namespaces):
                # Substitua o texto do 'tspan' pelo novo texto
                tspan.text = novo_texto

#### Definindo Função para Cálculo da Salinidade em g/l

Atualmente o código gera no report a salinidade em porcentagem de peso wt% (weight percent), porém a salinidade depende da densidade que é inversamente proporcional a temperatura, por isso é necessário o uso da função a seguir:

In [13]:
def salt_water_density(T, s):
    a0 = 9.999 * 10**2
    a1 = 2.034 * 10**-2
    a2 = -6.162 * 10**-3
    a3 = 2.261 * 10**-5
    a4 = -4.657 * 10**-8
    b0 = 8.020 * 10**2
    b1 = -2.001
    b2 = 1.677 * 10**-2
    b3 = -3.060 * 10**-5
    b4 = -1.613 * 10**-5

    result = (a0 + a1 * T + a2 * T**2 + a3 * T**3 + a4 * T**4 + 
              b0 * s + b1 * s * T + b2 * s * T**2 + 
              b3 * s * T**3 + b4 * s**2 * T**2)

    return result

#### Definindo cada a temperatura para cada Salinidade

In [14]:
# De cima para baixo no desenho e da esquerda para a direita, temos

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### HotHex - Cold side salinity (wt%) -> está presente na primeira e na segunda instância de salinidade, os valores são iguais em wt% mas distintos em g/L


# A temperatura da primeira instância (mais a esquerda) é a variável HotHex - Cold side outlet temperature (°C)

# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 1 (wt%)

T_css1 = float(report_filtrado['HotHex - Cold side outlet temperature (°C)'].iloc[0])
S_css1 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css1_g_l = "s " + str(round(S_css1 * salt_water_density(T_css1, S_css1), 2)) + " g/l"

# A temperatura da segunda instância (mais a direita) é a variável HotHex - Cold side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para HotHex - Cold side salinity 2 (wt%)

T_css2 = float(report_filtrado['HotHex - Cold side inlet temperature (°C)'].iloc[0])
S_css2 = float(report_filtrado['HotHex - Cold side salinity (wt%)'].iloc[0]/100)
css2_g_l = "s " + str(round(S_css2 * salt_water_density(T_css2, S_css2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Desal - Hot feedwater outlet salinity (wt%) -> terceira instância de salinidade

# A temperatura da terceira instância está na variável -> Desal - Hot feedwater outlet temperature (°C)

T_hfos = float(report_filtrado['Desal - Hot feedwater outlet temperature (°C)'].iloc[0])
S_hfos = float(report_filtrado['Desal - Hot feedwater outlet salinity (wt%)'].iloc[0]/100)
hfos_g_l = "s " + str(round(S_hfos * salt_water_density(T_hfos, S_hfos), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### ColdHex - Hot side salinity (wt%) -> está presente na quarta e quinta instância de salinidade

# A temperatura da quarta instância (mais para cima) é a variável ColdHex - Hot side outlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 1 (wt%)

T_hss1 = float(report_filtrado['ColdHex - Hot side outlet temperature (°C)'].iloc[0])
S_hss1 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss1_g_l = "s " + str(round(S_hss1 * salt_water_density(T_hss1, S_hss1), 2)) + " g/l"

# A temperatura da quinta instância (mais para baixo) é a variável ColdHex - Hot side inlet temperature (°C)
# No desenho, o nome dessa instância da salinidade foi alterado para ColdHex - Hot side salinity 2 (wt%)

T_hss2 = float(report_filtrado['ColdHex - Hot side inlet temperature (°C)'].iloc[0])
S_hss2 = float(report_filtrado['ColdHex - Hot side salinity (wt%)'].iloc[0]/100)
hss2_g_l = "s " + str(round(S_hss2 * salt_water_density(T_hss2, S_hss2), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### Brackish water salinity at the entrance of the regulating tank = -> está presente na sexta instância de salinidade

# A temperatura da sexta instância está na variável -> Brackish water temperature at the entrance of the regulating tank =

#T_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water temperature at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])
#S_bws = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'Brackish water salinity at the entrance of the regulating tank =']['Unnamed: 1'].iloc[0])/100
#bws_g_l = "s " + str(round(S_bws * salt_water_density(T_bws, S_bws), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#### FeedTank - Water salinity (wt%) -> está presente na sétima instância de salinidade

# A temperatura da sétima instância está na variável -> FeedTank - Water temperature (°C)

T_fts = float(report_filtrado['FeedTank - Water temperature (°C)'].iloc[0])
S_fts = float(report_filtrado['FeedTank - Water salinity (wt%)'].iloc[0]/100)
fts_g_l = "s " + str(round(S_fts * salt_water_density(T_fts, S_fts), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#### RO Module - Brine outlet salinity (wt%) -> está presente na oitava instância de salinidade
# A temperatura da oitava instância está na variável -> RO Feedtank - Temperature (°C)
T_ros = float(report_filtrado['RO Feedtank - Temperature (°C)'].iloc[0])
S_ros = float(report_filtrado['RO Module - Brine outlet salinity (wt%)'].iloc[0]/100)
ros_g_l = "s " + str(round(S_ros * salt_water_density(T_ros, S_ros), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# RO feedtank brackish water salinity =
ro_feedtank_brackish_water_salinity = float(entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'RO feedtank brackish water salinity =']['Unnamed: 1'].iloc[0])/100
# RO feedtank brackish water temperature =
ro_feedtank_brackish_water_temperature = entry_data_filtrado[entry_data_filtrado['Entry data:'] == 'RO feedtank brackish water temperature =']['Unnamed: 1'].iloc[0]

# RO feedtank brackish water salinity g/L
ro_feedtank_brackish_water_salinity_g_l = "s " + str(round(float(ro_feedtank_brackish_water_salinity) * salt_water_density(float(ro_feedtank_brackish_water_temperature), ro_feedtank_brackish_water_salinity), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# RO Feedtank - Salinity (wt%)

ro_feedtank_salinity = float(report_filtrado['RO Feedtank - Salinity (wt%)'].iloc[0])/100
ro_feedtank_temperature = report_filtrado['RO Feedtank - Temperature (°C)'].iloc[0]
ro_feedtank_salinity_g_l = "s " + str(round(float(ro_feedtank_salinity) * salt_water_density(float(ro_feedtank_temperature), ro_feedtank_salinity), 2)) + " g/l"

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# RO Module - Permeate salinity (wt%)
ro_module_permeate_salinity = float(report_filtrado['RO Module - Permeate salinity (wt%)'].iloc[0])/100
ro_module_permeate_salinity_g_l = "s " + str(round(ro_module_permeate_salinity * salt_water_density(float(report_filtrado['RO Feedtank - Temperature (°C)'].iloc[0]), ro_module_permeate_salinity), 2)) + " g/l"


# calculando o Salinity of the treated water

salinity_treated_water = float(report_filtrado_tempo_final['Overall - Salinity of the treated water (wt%)'].iloc[0]/100)
salinity_treated_water_temperature = 25 # assumindo temperatura ambiente de 25°C
salinity_treated_water_g_l = " " + str(round(salinity_treated_water * salt_water_density(salinity_treated_water_temperature,salinity_treated_water), 2)) + " g/l"
print("Salinity of the treated water:", salinity_treated_water)
print("Salinity of the treated water:", salinity_treated_water_g_l)

Salinity of the treated water: 9.2870252e-05
Salinity of the treated water:  0.09 g/l


#### Passando o Desal - Total water produced (kg) para litro -> vamos dividir o valor em kg por 0.998

In [15]:
#total_water_produced_kg = report_filtrado['Desal - Total water produced (kg)'].iloc[0]
#total_water_produced_l = str(round(total_water_produced_kg / 0.998, 2)) + " L"
total_water_produced_l = str(round(float(report_filtrado['Overall - Net water production (L)'].iloc[0]),2)) + " L"
print(total_water_produced_l)

629.16 L


#### Calculando o valor de (Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2

In [16]:
film_wall_temp = report_filtrado['Desal - Film wall temperature (°C)'].iloc[0]
gap_film_boundary_temp = report_filtrado['Desal - Gap film boundary temperature (°C)'].iloc[0]
average_temp = "T "+ str(round((film_wall_temp + gap_film_boundary_temp) / 2, 2)) + " °C"

#### Calculando o valor de Motor - Radiator in mass flow rate (kg/s) + Motor - Radiator bypass mass flow rate (kg/s)

In [17]:
motor_flow_rate_combined = "ṁ " + str(round(report_filtrado['Motor - Radiator in mass flow rate (kg/s)'].iloc[0] + report_filtrado['Motor - Radiator bypass mass flow rate (kg/s)'].iloc[0], 2)) + " kg/s"
print(motor_flow_rate_combined)

ṁ 2.49 kg/s


#### Calculando [RO Module - Brine outflow rate (kg/h)] - [RO Feedtank - Mass outflow rate (kg/h)]

In [18]:
#[RO Module - Brine outflow rate (kg/h)] - [FeedTank - Brackish water inlet flow rate (kg/h)]

ro_brine_calc = report_filtrado['RO Module - Brine outflow rate (kg/h)'].iloc[0] - report_filtrado['FeedTank - Brackish water inlet flow rate (kg/h)'].iloc[0]
#ro_brine_calc = "ṁ " + str(round(ro_brine_calc/3600, 2)) + " kg/s"
ro_brine_calc = "ṁ " + str(round(ro_brine_calc, 2)) + " kg/h"
print(ro_brine_calc)


ṁ 414.71 kg/h


#### Calculando [RO Feedtank - Mass outflow rate (kg/h)] - ([RO Module - Brine outflow rate (kg/h)] - [FeedTank - Brackish water inlet flow rate (kg/h)])

In [19]:
#[RO Feedtank - Mass outflow rate (kg/h)] - ([RO Module - Brine outflow rate (kg/h)] - [FeedTank - Brackish water inlet flow rate (kg/h)])

ro_feedtank_calc = float(report_filtrado['RO Feedtank - Mass outflow rate (kg/h)'].iloc[0]) - float(ro_brine_calc.split(" ")[1])
#ro_feedtank_calc = "ṁ " + str(round(float(ro_feedtank_calc/3600), 2)) + " kg/s"
ro_feedtank_calc = "ṁ " + str(round(float(ro_feedtank_calc), 2)) + " kg/h"
print(ro_feedtank_calc)

ṁ 202.09 kg/h


#### Calculando [RO Feedtank - Mass outflow rate (kg/h)] - [RO Module - Brine outflow rate (kg/h)]

In [20]:
# [RO Feedtank - Mass outflow rate (kg/h)] - [RO Module - Brine outflow rate (kg/h)]

ro_feedtank_brine_calc = float(report_filtrado['RO Feedtank - Mass outflow rate (kg/h)'].iloc[0]) - float(report_filtrado['RO Module - Brine outflow rate (kg/h)'].iloc[0])
#ro_feedtank_brine_calc = "ṁ " + str(round(float(ro_feedtank_brine_calc)/3600, 2)) + " kg/s"
ro_feedtank_brine_calc = "ṁ " + str(round(float(ro_feedtank_brine_calc), 2)) + " kg/h"
print(ro_feedtank_brine_calc)

ṁ 169.88 kg/h


##### Substituindo os Valores

In [21]:
for coluna, valor in report_filtrado.items():
    valor = valor.iloc[0]
    grandeza = ""
    if '(' in coluna and ')' in coluna:
        unidade = coluna[coluna.rfind('(') + 1:coluna.rfind(')')]
        unidade = unidade.strip()
    elif coluna == "SolarField - Efficiency" or coluna == "PVT - Thermal Efficiency" or coluna == "ETC - Efficiency" or coluna == "PVT - Electrical Efficiency" or coluna == "Motor - System efficiency":
        unidade = "%"
    else:
        unidade = ""
    if coluna == "PVT - Heat transfer rate (W)" or coluna == "SolarField - Heat transfer rate (W)" or coluna == "ETC - Heat transfer rate (W)" or coluna == "PVT - Electrical power output (W)" or coluna == "Electrolyzer - Heat transfer rate generated (W)" or coluna == "Electrolyzer - Power demand (W)":
        unidade = "kW"
        valor = "{:.2f}".format(float(valor/1000))
    if coluna == "HotHex - Heat transfer rate (W)" or coluna == "ColdHex - Heat transfer rate (W)":
        unidade = "kW"
        grandeza = "Q"
        valor = "{:.2f}".format(float(valor/1000))
    #if coluna == "RO Module - Brine outflow rate (kg/h)" or coluna == "FeedTank - Brackish water inlet flow rate (kg/h)" or coluna == "RO Feedtank - Mass outflow rate (kg/h)":
    #    unidade = "kg/s"
    #    grandeza = "ṁ"
    #    valor = "{:.2f}".format(float(valor/3600))
    if unidade == '°C':
        grandeza = "T"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'Kg/h' or unidade == 'kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))
    valor = "{:.2f}".format(float(valor))
    valor_novo = grandeza + " " + str(valor) + " " + unidade
    print(valor_novo)
    substituir_texto_por_label(coluna, valor_novo)

 11088.00 s
 629.16 L
ṁ 32.21 kg/h
ṁ 32.21 kg/h
Q 4.47 kW
T 31.92 °C
T 37.30 °C
ṁ 567.79 kg/h
 2.14 wt%
 2.03 wt%
T 30.06 °C
 2.03 wt%
Q 4.94 kW
T 77.28 °C
T 77.72 °C
T 77.70 °C
T 36.63 °C
ṁ 600.00 kg/h
T 70.46 °C
 952.20 W/m²
 34.16 %
T 76.10 °C
T 79.17 °C
T 79.06 °C
T 76.27 °C
 2.03 wt%
T 36.63 °C
T 51.53 °C
T 53.70 °C
T 76.27 °C
T 69.49 °C
ṁ 0.34 kg/s
 -8.62 kW
 4.06 kW
 23.69 %
 11.05 kWh
ṁ 0.34 kg/s
 55.11 %
T 76.27 °C
T 79.03 °C
ṁ 0.23 kg/s
T 76.27 °C
 40.05 %
 0.49 kW
 13.73 %
 1.35 kWh
T 77.96 °C
ṁ 0.23 kg/s
 1.43 kW
 2.34 kW
 3.90 kW
T 77.72 °C
ṁ 2.99 kg/s
ṁ 2.49 kg/s
T 78.01 °C
ṁ 0.09 kg/s
ṁ 2.40 kg/s
ṁ 2.04 kg/s
ṁ 4.53 kg/s
T 77.23 °C
T 76.59 °C
T 57.69 °C
 13.25 kW
 34.92 %
 61.60 kWh
 18.33 L
 5.95 L/h
 240.98 g/kWh
 0.41 Nm³/h
 1.14 Nm³
ṁ 3.42 kg/h
 9.59 kg
 65.58 %
 1.05 kW
 2.28 kW
 6.20 kWh
T 25.00 °C
 1.83 wt%
ṁ 446.92 kg/h
ṁ 616.80 kg/h
 1.32 wt%
 51.24 bar
 0.01 wt%
 3.08 kWh
ṁ 0.50 kg/s
ṁ 76.27 kg/s
T 76.27 °C
 64.72 kWh
 27.42 kWh
 0.01 wt%
 6.8

In [22]:
for linha in entry_data_filtrado.itertuples():
    grandeza = ""
    label = linha[1].rstrip(' ')
    valor = linha[2]
    valor = "{:.2f}".format(float(valor))
    unidade = linha[3].strip()
    if unidade == '°C':
        grandeza = "T"
        if label == "Electrolyzer cell temperature =":
            grandeza = ""
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/s':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
    if unidade == 'kg/h' or unidade == 'Kg/h':
        grandeza = "ṁ"
        valor = "{:.2f}".format(float(valor))
        #if label == "Brine flow rate out of the regulating tank =":
        #    grandeza = "ṁ"
        #    unidade = "kg/s"
        #    valor = "{:.2f}".format(float(valor)/3600)
    if unidade == 'g/L':
        grandeza = "S"
        valor = "{:.2f}".format(float(valor))

    valor_novo = grandeza + " " + str(valor) + ' ' + unidade
    print(label, valor_novo)
    substituir_texto_por_label(label, str(valor_novo))

Cooling heat exchanger flow rate = ṁ 2000.00 kg/h
Temperature at the entrance of the cold side = T 30.00 °C
Brine flow rate out of the regulating tank = ṁ 0.00 kg/h
Motor power demanded/generated =  20.00 kW
Electrolyzer cell temperature =  40.00 °C
Cathode pressure =  70.00 bar
RO pump power =  1.00 kW
RO feedtank brackish water temperature = T 25.00 °C
RO feedtank brackish water salinity =  1.00 wt%


In [23]:
substituir_texto_por_label("HotHex - Cold side salinity 1 (wt%)", css1_g_l)
substituir_texto_por_label("HotHex - Cold side salinity 2 (wt%)", css2_g_l)
substituir_texto_por_label("Desal - Hot feedwater outlet salinity (wt%)", hfos_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 1 (wt%)", hss1_g_l)
substituir_texto_por_label("ColdHex - Hot side salinity 2 (wt%)", hss2_g_l)
#substituir_texto_por_label("Brackish water salinity at the entrance of the regulating tank =", bws_g_l)
substituir_texto_por_label("FeedTank - Water salinity (wt%)", fts_g_l)
substituir_texto_por_label("RO Module - Brine outlet salinity (wt%)", ros_g_l)
substituir_texto_por_label("RO feedtank brackish water salinity =", ro_feedtank_brackish_water_salinity_g_l)
substituir_texto_por_label("Motor - Radiator in mass flow rate (kg/s) + Motor - Radiator bypass mass flow rate (kg/s)", motor_flow_rate_combined)

# correção do valor de total de água produzida
substituir_texto_por_label("Desal - Total water produced (kg)", total_water_produced_l)

# correção do valor de temperatura média
substituir_texto_por_label("(Desal - Film wall temperature (°C)+Desal - Gap film boundary temperature (°C))/2", average_temp)

# valor do [RO Module - Brine outflow rate (kg/h)] - [FeedTank - Brackish water inlet flow rate (kg/h)]
substituir_texto_por_label("[RO Module - Brine outflow rate (kg/h)] - [FeedTank - Brackish water inlet flow rate (kg/h)]", ro_brine_calc)

# valor do [RO Feedtank - Mass outflow rate (kg/h)] - ([RO Module - Brine outflow rate (kg/h)] - [FeedTank - Brackish water inlet flow rate (kg/h)])
substituir_texto_por_label("[RO Feedtank - Mass outflow rate (kg/h)] - ([RO Module - Brine outflow rate (kg/h)] - [FeedTank - Brackish water inlet flow rate (kg/h)])", ro_feedtank_calc)

# valor RO Feedtank - Salinity (wt%)
substituir_texto_por_label("RO Feedtank - Salinity (wt%)", ro_feedtank_salinity_g_l)

# RO Module - Permeate salinity (wt%)
substituir_texto_por_label("RO Module - Permeate salinity (wt%)", ro_module_permeate_salinity_g_l)

# [RO Feedtank - Mass outflow rate (kg/h)] - [RO Module - Brine outflow rate (kg/h)]
substituir_texto_por_label("[RO Feedtank - Mass outflow rate (kg/h)] - [RO Module - Brine outflow rate (kg/h)]", ro_feedtank_brine_calc)

# Substituindo os valores de overall do tempo final
substituir_texto_por_label("Overall - Net electricity generation (kWh)", net_electricity_generation)
substituir_texto_por_label("Overall - Net water production - ft - (L)", net_water_production)
substituir_texto_por_label("Overall - Net heat recovered (kWh)", net_heat_recovered)
substituir_texto_por_label("Overall - Salinity of the treated water (wt%)", salinity_treated_water_g_l)

##### Salvando o arquivo

In [24]:
# Salvar as alterações no arquivo SVG
tree.write('Diag_geral_modificado.svg')